In [ ]:
!pip install anvil-uplink #Buradaki pip installı yapıp ardından yeniden bağlantıyı başlatın 2.sefer başlattığınızda gereklilikler yüklendiğinden bu bloğu çalıştırmadan devam etmelisiniz 

In [ ]:
import anvil.server

anvil.server.connect("server_5YNXKRULTK7Z6Z7M23NBGINW-OFYEENN2PR7A5ESP")


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline

tokenizer = AutoTokenizer.from_pretrained("LugatitTurk/LugatitBert")
model = AutoModelForSequenceClassification.from_pretrained("LugatitTurk/LugatitBert")

nlp = pipeline("text-classification", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/755k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

In [ ]:


tokenizer2 = AutoTokenizer.from_pretrained("LugatitTurk/LugatitHaberler")
model2 = AutoModelForSequenceClassification.from_pretrained("LugatitTurk/LugatitHaberler")


nlp2 = pipeline("text-classification", model=model2, tokenizer=tokenizer2)



tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/755k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
#  'LABEL_0': 'dunya ',
#  'LABEL_1': 'ekonomi ',
#  'LABEL_2': 'kultur ',
#  'LABEL_3': 'saglik ',
#  'LABEL_4': 'siyaset ',
#  'LABEL_5': 'spor ',
#  'LABEL_6': 'diğer '
#HABER KISMI

#'LABEL_0': Diğer
#'LABEL_1': Saldırgan/Zorbalık
#'LABEL_2': Yönlendirici Şiddet

In [ ]:
@anvil.server.callable
def predict_tweet(metin):

  inputs = tokenizer(metin, return_tensors="pt")
  outputs = model(**inputs)
  first_three_probs = outputs[0][0][:3]
  probabilities = F.softmax(first_three_probs, dim=0)
  percentages = probabilities * 100
  result = percentages.detach().numpy()

  if( result[0] < result[2] and result[1] < result[2] ):
    inputs = tokenizer2(metin, return_tensors="pt")
    outputs = model2(**inputs)
    first_three_probs = outputs[0][0]
    probabilities = F.softmax(first_three_probs, dim=0)
    percentages = probabilities * 100
    result2 = percentages.detach().numpy()
    return [result,result2]
  else :
     return result


In [ ]:
import pandas as pd
import io

@anvil.server.callable
def process_uploaded_file(file):
  byteStream = file.get_bytes()
  excel_stream = io.BytesIO(byteStream)
  df = pd.read_excel(excel_stream)
  num1 = 0
  num2 = 0
  num3 = 0
  haber1Kategori3 = 0
  haber2Kategori3 = 0
  haber3Kategori3 = 0
  haber4Kategori3 = 0
  haber5Kategori3 = 0
  haber6Kategori3 = 0
  haber7Kategori3 = 0
  for i in range (len(df)):
    resultObject = nlp(df['Tweet'][i])
    if(str(resultObject[0]['label']) == "LABEL_0"):
      num1  = num1 + 1;
    elif(str(resultObject[0]['label']) == "LABEL_1"):
      num2  = num2 + 1;
    elif(str(resultObject[0]['label']) == "LABEL_2"):
      num3  = num3 + 1;

  for i in range (len(df)):
    resultObject = nlp2(df['Tweet'][i])
    resultObjectKategori = nlp(df['Tweet'][i])
    if(str(resultObject[0]['label']) == "LABEL_0" and (str(resultObjectKategori[0]['label']) == "LABEL_2") ):
      haber1Kategori3  = haber1Kategori3 + 1;
    elif(str(resultObject[0]['label']) == "LABEL_1" and (str(resultObjectKategori[0]['label']) == "LABEL_2") ):
      haber2Kategori3  = haber2Kategori3 + 1;
    elif(str(resultObject[0]['label']) == "LABEL_2" and (str(resultObjectKategori[0]['label']) == "LABEL_2")):
      haber3Kategori3  = haber3Kategori3 + 1;
    elif(str(resultObject[0]['label']) == "LABEL_3" and (str(resultObjectKategori[0]['label']) == "LABEL_2") ):
      haber4Kategori3  = haber4Kategori3 + 1;
    elif(str(resultObject[0]['label']) == "LABEL_4" and (str(resultObjectKategori[0]['label']) == "LABEL_2")):
      haber5Kategori3  = haber5Kategori3 + 1;
    elif(str(resultObject[0]['label']) == "LABEL_5" and (str(resultObjectKategori[0]['label']) == "LABEL_2")):
      haber6Kategori3  = haber6Kategori3 + 1;
    elif(str(resultObject[0]['label']) == "LABEL_6" and (str(resultObjectKategori[0]['label']) == "LABEL_2")):
      haber7Kategori3  = haber7Kategori3 + 1;


  return [[num1,num2,num3],[haber1Kategori3,haber2Kategori3,haber3Kategori3,haber4Kategori3,haber5Kategori3,haber6Kategori3,haber7Kategori3]]

In [ ]:
#Buradaki kodların metod ismiyle anvilden gerektiği zaman çağrı yapılmaktadır. Burada upload modülünde dikkat edilmesi gereken huss koddan anlaşılacağı üzere 
#Tweet sütunu oluşturup bunun altına tweet metinlerini hücre hücre eklemelisiniz.
